# Oracle Metadata Knowledge Graph - Exploration

This notebook demonstrates how to interact with the Neo4j knowledge graph containing Oracle metadata.

## Setup

In [ ]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
import pandas as pd

load_dotenv()

# Connect to Neo4j
uri = os.getenv('NEO4J_URI', 'bolt://localhost:7687')
user = os.getenv('NEO4J_USER', 'neo4j')
password = os.getenv('NEO4J_PASSWORD', 'password123')

driver = GraphDatabase.driver(uri, auth=(user, password))
print("✓ Connected to Neo4j")

## Helper Functions

In [ ]:
def run_query(query, params=None):
    """Run a Cypher query and return results as DataFrame"""
    with driver.session() as session:
        result = session.run(query, params or {})
        return pd.DataFrame([dict(record) for record in result])

def print_query(query, params=None):
    """Run query and display results"""
    df = run_query(query, params)
    display(df)
    return df

## 1. Basic Statistics

In [ ]:
# Count nodes by type
query = """
MATCH (n)
RETURN labels(n)[0] as NodeType, count(n) as Count
ORDER BY Count DESC
"""
print_query(query)

## 2. List All Tables

In [ ]:
query = """
MATCH (t:Table)
RETURN t.name as TableName, t.owner as Owner, t.num_rows as RowCount
ORDER BY t.name
"""
print_query(query)

## 3. Table Details with Column Count

In [ ]:
query = """
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WITH t, count(c) as ColumnCount
RETURN t.name as TableName, ColumnCount, t.num_rows as RowCount
ORDER BY ColumnCount DESC
"""
print_query(query)

## 4. Explore a Specific Table

In [ ]:
# Change table name as needed
table_name = 'EMPLOYEES'

query = """
MATCH (t:Table {name: $table_name})-[:HAS_COLUMN]->(c:Column)
RETURN c.name as ColumnName, c.data_type as DataType, 
       c.nullable as Nullable, c.data_length as Length
ORDER BY c.position
"""
print_query(query, {'table_name': table_name})

## 5. Foreign Key Relationships

In [ ]:
query = """
MATCH (t1:Table)-[fk:HAS_FOREIGN_KEY]->(t2:Table)
RETURN t1.name as FromTable, fk.constraint_name as ConstraintName, 
       fk.columns as Columns, t2.name as ToTable
ORDER BY t1.name
"""
print_query(query)

## 6. Indexes by Table

In [ ]:
query = """
MATCH (t:Table)-[:HAS_INDEX]->(i:Index)
RETURN t.name as TableName, i.name as IndexName, 
       i.uniqueness as Uniqueness, i.columns as Columns
ORDER BY t.name, i.name
"""
print_query(query)

## 7. Find Tables with Most Foreign Keys

In [ ]:
query = """
MATCH (t:Table)-[fk:HAS_FOREIGN_KEY]->()
WITH t, count(fk) as FKCount
RETURN t.name as TableName, FKCount
ORDER BY FKCount DESC
LIMIT 10
"""
print_query(query)

## 8. Data Type Distribution

In [ ]:
query = """
MATCH (c:Column)
RETURN c.data_type as DataType, count(c) as Count
ORDER BY Count DESC
"""
df = print_query(query)

# Visualize
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.bar(df['DataType'], df['Count'])
plt.xlabel('Data Type')
plt.ylabel('Count')
plt.title('Column Data Type Distribution')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Table Relationship Graph

In [ ]:
# Get all foreign key relationships
query = """
MATCH (t1:Table)-[:HAS_FOREIGN_KEY]->(t2:Table)
RETURN t1.name as source, t2.name as target
"""
df = run_query(query)

# Create network graph
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()
for _, row in df.iterrows():
    G.add_edge(row['source'], row['target'])

plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, k=2, iterations=50)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=3000, font_size=10, font_weight='bold',
        arrows=True, arrowsize=20, edge_color='gray')
plt.title('Table Foreign Key Relationships')
plt.tight_layout()
plt.show()

## 10. Natural Language Query with LangChain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

# Initialize
graph = Neo4jGraph(
    url=uri,
    username=user,
    password=password
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4",
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True
)

# Ask a question
question = "What tables have foreign keys to the DEPARTMENTS table?"
result = chain.invoke({"query": question})
print(f"\nQuestion: {question}")
print(f"Answer: {result['result']}")

## Cleanup

In [ ]:
driver.close()
print("✓ Connection closed")